# Naive Bayes Classification

In [2]:
import pre_ml
import mlpipeline as pipeline
import pandas as pd
import numpy as np
import sklearn

In [3]:
df = pre_ml.data()

Requesting data to datos abiertos Mexico
Getting zip raw data into directory, will delete soon
Raw data deleted. If you specified filename, clean data will be saved in data directory


In [4]:
df.head(5)

,entidad_res,municipio_res,CVE_MUN,embarazo,edad,diabetes,epoc,asma,inmusupr,hipertension,...,obesidad,tabaquismo,pobreza,Densidad_pob,medicos,camas_hosp,enfermeras,hospitalizado,muertos,covid_grave
0,25,6,25006,0.0,27,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,31.2,152.715706,2.637944,10.489463,30.502528,0,0,0
1,27,4,27004,0.0,44,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,36.3,430.537057,5.448810,13.926240,41.575909,0,0,0
2,27,5,27005,0.0,52,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,63.6,283.191451,3.401376,2.436121,8.319582,1,0,1
3,15,58,15058,0.0,55,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,37.4,17954.252292,0.810012,2.870259,10.248409,0,0,0
4,2,4,2004,0.0,47,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,31.1,1449.318075,0.726447,4.889549,11.701390,1,1,1


Check data:

In [5]:
df.describe()

,entidad_res,municipio_res,CVE_MUN,embarazo,edad,diabetes,epoc,asma,inmusupr,hipertension,...,obesidad,tabaquismo,pobreza,Densidad_pob,medicos,camas_hosp,enfermeras,hospitalizado,muertos,covid_grave
count,93435.000000,93435.000000,93435.000000,93435.000000,93435.000000,93435.000000,93435.000000,93435.000000,93435.000000,93435.000000,...,93435.000000,93435.000000,93435.000000,93435.000000,93435.000000,93435.000000,93435.000000,93435.000000,93435.000000,93435.000000
mean,15.057537,36.730593,15094.267919,0.006336,46.241034,0.174282,0.019693,0.029026,0.015701,0.208070,...,0.203885,0.082603,36.873778,4743.009499,2.770005,11.096877,29.604093,0.348981,0.108814,0.360143
std,8.105571,52.420438,8121.567737,0.079347,16.014916,0.379353,0.138943,0.167879,0.124316,0.405929,...,0.402887,0.275282,14.105819,5873.026693,2.008568,10.075898,24.243500,0.476650,0.311407,0.480044
min,1.000000,1.000000,1001.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,3.200000,0.446811,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.000000,6.000000,9007.000000,0.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,28.700000,254.086894,1.506770,5.474526,15.946373,0.000000,0.000000,0.000000
50%,15.000000,15.000000,15024.000000,0.000000,45.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,34.500000,1814.604740,2.248080,8.238176,21.720906,0.000000,0.000000,0.000000
75%,21.000000,50.000000,21119.000000,0.000000,57.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,41.200000,8183.315370,3.759853,13.572164,36.168421,1.000000,0.000000,1.000000
max,32.000000,999.000000,32057.000000,1.000000,120.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,99.100000,17954.252292,50.068722,297.467112,559.591596,1.000000,1.000000,1.000000


Drop outlier observations for age

In [6]:
df = df.drop(df[df.edad > (df.edad.mean()+3*df.edad.std())].index)

Re-scale poverty measurement

In [7]:
df['pobreza'] = df['pobreza'] / 100

Probability of dying:

In [9]:
deaths = df[df['muertos']==1]['muertos'].count()
no_deaths = df[df['muertos']==0]['muertos'].count()
cases = df['muertos'].count()

# Probability of dying for COVID-19
p_rip = [deaths / cases,
        no_deaths / cases]

print("P(dying) = {:10.4f}\nP(not dying) = {:10.4f}".format(p_rip[0],p_rip[1]))

P(dying) =     0.1086
P(not dying) =     0.8914


Split data into train/test sets:

In [10]:
train, test = pipeline.split_data(df.loc[:,'embarazo':], .20, seed=0)

Check for balance in train/test:

In [11]:
train['muertos'].value_counts()

0    66566
1     8094
Name: muertos, dtype: int64

In [12]:
train['hospitalizado'].value_counts()

0    48608
1    26052
Name: hospitalizado, dtype: int64

In [13]:
test['muertos'].value_counts()

0    16624
1     2041
Name: muertos, dtype: int64

In [14]:
test['hospitalizado'].value_counts()

0    12166
1     6499
Name: hospitalizado, dtype: int64

In [15]:
train.describe()

,embarazo,edad,diabetes,epoc,asma,inmusupr,hipertension,cardiovascular,obesidad,tabaquismo,pobreza,Densidad_pob,medicos,camas_hosp,enfermeras,hospitalizado,muertos,covid_grave
count,74660.000000,74660.000000,74660.000000,74660.000000,74660.000000,74660.000000,74660.000000,74660.000000,74660.000000,74660.000000,74660.000000,74660.000000,74660.000000,74660.000000,74660.000000,74660.000000,74660.000000,74660.000000
mean,0.006389,46.201326,0.173841,0.019756,0.028891,0.015617,0.207822,0.025837,0.203523,0.082681,0.368844,4750.381673,2.771698,11.100244,29.591547,0.348942,0.108411,0.360139
std,0.079676,15.967045,0.378976,0.139162,0.167501,0.123991,0.405752,0.158650,0.402621,0.275402,0.141022,5873.716516,2.012931,10.098444,24.226843,0.476639,0.310901,0.480044
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032000,0.446811,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.287000,254.086894,1.506770,5.474526,15.903346,0.000000,0.000000,0.000000
50%,0.000000,45.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.345000,1814.604740,2.248080,8.238176,21.720906,0.000000,0.000000,0.000000
75%,0.000000,57.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.412000,8183.315370,3.759853,13.672404,36.168421,1.000000,0.000000,1.000000
max,1.000000,94.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.991000,17954.252292,50.068722,297.467112,559.591596,1.000000,1.000000,1.000000


In [16]:
test.describe()

,embarazo,edad,diabetes,epoc,asma,inmusupr,hipertension,cardiovascular,obesidad,tabaquismo,pobreza,Densidad_pob,medicos,camas_hosp,enfermeras,hospitalizado,muertos,covid_grave
count,18665.000000,18665.000000,18665.000000,18665.000000,18665.000000,18665.000000,18665.000000,18665.000000,18665.000000,18665.000000,18665.000000,18665.000000,18665.000000,18665.000000,18665.000000,18665.000000,18665.000000,18665.000000
mean,0.006161,46.098312,0.176266,0.018805,0.029574,0.015751,0.208090,0.025181,0.206322,0.082400,0.368470,4710.340932,2.761847,11.070872,29.613717,0.348192,0.109349,0.358961
std,0.078254,15.767218,0.381056,0.135840,0.169414,0.124516,0.405952,0.156678,0.404675,0.274981,0.141023,5869.689261,1.991319,9.976615,24.270544,0.476410,0.312085,0.479708
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032000,0.446811,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.287000,253.234919,1.506770,5.474526,16.131462,0.000000,0.000000,0.000000
50%,0.000000,45.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.346000,1813.733810,2.248080,8.238176,21.720906,0.000000,0.000000,0.000000
75%,0.000000,57.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.412000,8183.315370,3.772233,13.572164,35.940020,1.000000,0.000000,1.000000
max,1.000000,94.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.976000,17954.252292,50.068722,297.467112,559.591596,1.000000,1.000000,1.000000


In [65]:
feats = ['embarazo', 'edad', 'diabetes', 'epoc', 'asma', 'inmusupr', 'hipertension', 'cardiovascular',
            'obesidad', 'tabaquismo', 'pobreza', 'Densidad_pob', 'medicos', 'camas_hosp', 'enfermeras']

feats_train = train.loc[:, feats]
target1_train = train.loc[:, 'muertos']
target2_train = train.loc[:, 'hospitalizado']
target3_train = train.loc[:, 'covid_grave']

feats_test = test.loc[:, feats]
target1_test = test.loc[:, 'muertos']
target2_test = test.loc[:, 'hospitalizado']
target3_test = test.loc[:, 'covid_grave']

### 1.1 Gaussian Naive Bayes (GNB) T1 (Deaths)

Train for  Gaussian distribution:

In [28]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB

params = {}

nbg = GridSearchCV(estimator=GaussianNB(), param_grid=params, cv=10 ,
                   return_train_score=True,
                   scoring=['accuracy', 'precision', 'recall'], 
                   refit='accuracy')

nbg.fit(feats_train, target1_train)
cv_results_nbg = pd.DataFrame(nbg.cv_results_)
cols = ['params', 'mean_test_accuracy', 'mean_test_precision', 'mean_test_recall']

cv_results_nbg[cols]

,params,mean_test_accuracy,mean_test_precision,mean_test_recall
0,{},0.855304,0.295961,0.242649


In [31]:
predicted_target1 = nbg.predict(feats_test)

print('Confusion matrix')
print(sklearn.metrics.confusion_matrix(target1_test, predicted_target1))

print('Accuracy score')
print(sklearn.metrics.accuracy_score(target1_test, predicted_target1))

print('Precision score')
print(sklearn.metrics.precision_score(target1_test, predicted_target1))
      
print('Recall score')
print(sklearn.metrics.recall_score(target1_test, predicted_target1))
      
print('F1 score')
print(sklearn.metrics.f1_score(target1_test, predicted_target1))

Confusion matrix
[[15518  1106]
 [ 1500   541]]
Accuracy score
0.8603803911063488
Precision score
0.3284760170006072
Recall score
0.26506614404703577
F1 score
0.2933839479392624


#### Upsampling

In [56]:
from sklearn.utils import resample
negatives = train[train.muertos == 0]
positives = train[train.muertos == 1]

upsample_positives = resample(positives,
                              replace=True, 
                              n_samples=len(negatives) // 2, 
                              random_state=0)
upsampled_train = pd.concat([negatives, upsample_positives])

feats_train_up = upsampled_train.loc[:, feats]
target1_train_up = upsampled_train.loc[:, 'muertos']

In [57]:
upsampled_train.muertos.value_counts()

0    66566
1    33283
Name: muertos, dtype: int64

In [58]:
nbg = GridSearchCV(estimator=GaussianNB(), param_grid=params, cv=10 ,
                   return_train_score=True,
                   scoring=['accuracy', 'precision', 'recall'], 
                   refit='accuracy')

nbg.fit(feats_train_up, target1_train_up)
cv_results_nbg = pd.DataFrame(nbg.cv_results_)
cols = ['params', 'mean_test_accuracy', 'mean_test_precision', 'mean_test_recall']

cv_results_nbg[cols]

,params,mean_test_accuracy,mean_test_precision,mean_test_recall
0,{},0.716342,0.598583,0.452724


In [59]:
predicted_target1 = nbg.predict(feats_test)

print('Confusion matrix')
print(sklearn.metrics.confusion_matrix(target1_test, predicted_target1))

print('Accuracy score')
print(sklearn.metrics.accuracy_score(target1_test, predicted_target1))

print('Precision score')
print(sklearn.metrics.precision_score(target1_test, predicted_target1))
      
print('Recall score')
print(sklearn.metrics.recall_score(target1_test, predicted_target1))
      
print('F1 score')
print(sklearn.metrics.f1_score(target1_test, predicted_target1))

Confusion matrix
[[14171  2453]
 [ 1055   986]]
Accuracy score
0.812054647736405
Precision score
0.2867112532712998
Recall score
0.48309652131308184
F1 score
0.3598540145985402


#### Undersampling

In [41]:
undersample_negatives = resample(negatives,
                                 replace=True, # sample with replacement
                                 n_samples=len(positives), # match number in majority class
                                 random_state=0)
undersample_train = pd.concat([positives, undersample_negatives])

feats_train_und = undersample_train.loc[:, feats]
target1_train_und = undersample_train.loc[:, 'muertos']

In [43]:
undersample_train.muertos.value_counts()

1    8094
0    8094
Name: muertos, dtype: int64

In [60]:
nbg = GridSearchCV(estimator=GaussianNB(), param_grid=params, cv=10 ,
                   return_train_score=True,
                   scoring=['accuracy', 'precision', 'recall'], 
                   refit='accuracy')

nbg.fit(feats_train_und, target1_train_und)
cv_results_nbg = pd.DataFrame(nbg.cv_results_)
cols = ['params', 'mean_test_accuracy', 'mean_test_precision', 'mean_test_recall']

cv_results_nbg[cols]

,params,mean_test_accuracy,mean_test_precision,mean_test_recall
0,{},0.674451,0.720415,0.570299


In [61]:
predicted_target1 = nbg.predict(feats_test)

print('Confusion matrix')
print(sklearn.metrics.confusion_matrix(target1_test, predicted_target1))

print('Accuracy score')
print(sklearn.metrics.accuracy_score(target1_test, predicted_target1))

print('Precision score')
print(sklearn.metrics.precision_score(target1_test, predicted_target1))
      
print('Recall score')
print(sklearn.metrics.recall_score(target1_test, predicted_target1))
      
print('F1 score')
print(sklearn.metrics.f1_score(target1_test, predicted_target1))

Confusion matrix
[[13178  3446]
 [  830  1211]]
Accuracy score
0.7709081167961425
Precision score
0.2600386514923771
Recall score
0.5933365997060265
F1 score
0.3616004777545536


#### SMOTE

In [69]:
from imblearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import KFold

pipeline = make_pipeline((SMOTE(random_state=0)), GaussianNB())

kf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

nbg = GridSearchCV(estimator=pipeline, param_grid=params, cv=kf ,
                   return_train_score=True,
                   scoring=['accuracy', 'precision', 'recall'], 
                   refit='accuracy')

nbg.fit(feats_train, target1_train)
cv_results_nbg = pd.DataFrame(nbg.cv_results_)
cols = ['params', 'mean_test_accuracy', 'mean_test_precision', 'mean_test_recall']

cv_results_nbg[cols]

,params,mean_test_accuracy,mean_test_precision,mean_test_recall
0,{},0.723725,0.233318,0.677251


In [70]:
predicted_target1 = nbg.predict(feats_test)

print('Confusion matrix')
print(sklearn.metrics.confusion_matrix(target1_test, predicted_target1))

print('Accuracy score')
print(sklearn.metrics.accuracy_score(target1_test, predicted_target1))

print('Precision score')
print(sklearn.metrics.precision_score(target1_test, predicted_target1))
      
print('Recall score')
print(sklearn.metrics.recall_score(target1_test, predicted_target1))
      
print('F1 score')
print(sklearn.metrics.f1_score(target1_test, predicted_target1))

Confusion matrix
[[12273  4351]
 [  630  1411]]
Accuracy score
0.7331368872220734
Precision score
0.24488024991322457
Recall score
0.6913277804997551
F1 score
0.36165577342047933


### 1.2 GNB T2 (Hospitalization)


In [62]:
nbg = GridSearchCV(estimator=GaussianNB(), param_grid=params, cv=10 ,
                   return_train_score=True,
                   scoring=['accuracy', 'precision', 'recall'], 
                   refit='accuracy')

nbg.fit(feats_train, target2_train)
cv_results_nbg = pd.DataFrame(nbg.cv_results_)
cols = ['params', 'mean_test_accuracy', 'mean_test_precision', 'mean_test_recall']

cv_results_nbg[cols]

,params,mean_test_accuracy,mean_test_precision,mean_test_recall
0,{},0.705585,0.606167,0.446607


In [63]:
predicted_target2 = nbg.predict(feats_test)

print('Confusion matrix')
print(sklearn.metrics.confusion_matrix(target2_test, predicted_target2))

print('Accuracy score')
print(sklearn.metrics.accuracy_score(target2_test, predicted_target2))

print('Precision score')
print(sklearn.metrics.precision_score(target2_test, predicted_target2))
      
print('Recall score')
print(sklearn.metrics.recall_score(target2_test, predicted_target2))
      
print('F1 score')
print(sklearn.metrics.f1_score(target2_test, predicted_target2))

Confusion matrix
[[10291  1875]
 [ 3579  2920]]
Accuracy score
0.7077953388695419
Precision score
0.6089676746611054
Recall score
0.4492998922911217
F1 score
0.5170887196741634


#### SMOTE

In [71]:
pipeline = make_pipeline((SMOTE(random_state=0)), GaussianNB())

kf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

nbg = GridSearchCV(estimator=pipeline, param_grid=params, cv=kf ,
                   return_train_score=True,
                   scoring=['accuracy', 'precision', 'recall'], 
                   refit='accuracy')

nbg.fit(feats_train, target2_train)
cv_results_nbg = pd.DataFrame(nbg.cv_results_)
cols = ['params', 'mean_test_accuracy', 'mean_test_precision', 'mean_test_recall']

cv_results_nbg[cols]

,params,mean_test_accuracy,mean_test_precision,mean_test_recall
0,{},0.704049,0.577317,0.56734


In [72]:
predicted_target2 = nbg.predict(feats_test)

print('Confusion matrix')
print(sklearn.metrics.confusion_matrix(target2_test, predicted_target2))

print('Accuracy score')
print(sklearn.metrics.accuracy_score(target2_test, predicted_target2))

print('Precision score')
print(sklearn.metrics.precision_score(target2_test, predicted_target2))
      
print('Recall score')
print(sklearn.metrics.recall_score(target2_test, predicted_target2))
      
print('F1 score')
print(sklearn.metrics.f1_score(target2_test, predicted_target2))

Confusion matrix
[[9521 2645]
 [2810 3689]]
Accuracy score
0.7077417626573801
Precision score
0.5824123776444585
Recall score
0.567625788582859
F1 score
0.5749240240006234


### 1.3 GNB T3 (Hospitalization/Death)

In [66]:
nbg = GridSearchCV(estimator=GaussianNB(), param_grid=params, cv=10 ,
                   return_train_score=True,
                   scoring=['accuracy', 'precision', 'recall'], 
                   refit='accuracy')

nbg.fit(feats_train, target3_train)
cv_results_nbg = pd.DataFrame(nbg.cv_results_)
cols = ['params', 'mean_test_accuracy', 'mean_test_precision', 'mean_test_recall']

cv_results_nbg[cols]

,params,mean_test_accuracy,mean_test_precision,mean_test_recall
0,{},0.707929,0.629515,0.459723


In [67]:
predicted_target3 = nbg.predict(feats_test)

print('Confusion matrix')
print(sklearn.metrics.confusion_matrix(target3_test, predicted_target3))

print('Accuracy score')
print(sklearn.metrics.accuracy_score(target3_test, predicted_target3))

print('Precision score')
print(sklearn.metrics.precision_score(target3_test, predicted_target3))
      
print('Recall score')
print(sklearn.metrics.recall_score(target3_test, predicted_target3))
      
print('F1 score')
print(sklearn.metrics.f1_score(target3_test, predicted_target3))

Confusion matrix
[[10159  1806]
 [ 3588  3112]]
Accuracy score
0.71100991159925
Precision score
0.6327775518503457
Recall score
0.4644776119402985
F1 score
0.5357204338096058


#### SMOTE

In [73]:
pipeline = make_pipeline((SMOTE(random_state=0)), GaussianNB())

kf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

nbg = GridSearchCV(estimator=pipeline, param_grid=params, cv=kf ,
                   return_train_score=True,
                   scoring=['accuracy', 'precision', 'recall'], 
                   refit='accuracy')

nbg.fit(feats_train, target3_train)
cv_results_nbg = pd.DataFrame(nbg.cv_results_)
cols = ['params', 'mean_test_accuracy', 'mean_test_precision', 'mean_test_recall']

cv_results_nbg[cols]

,params,mean_test_accuracy,mean_test_precision,mean_test_recall
0,{},0.708416,0.602032,0.5618


In [74]:
predicted_target3 = nbg.predict(feats_test)

print('Confusion matrix')
print(sklearn.metrics.confusion_matrix(target3_test, predicted_target3))

print('Accuracy score')
print(sklearn.metrics.accuracy_score(target3_test, predicted_target3))

print('Precision score')
print(sklearn.metrics.precision_score(target3_test, predicted_target3))
      
print('Recall score')
print(sklearn.metrics.recall_score(target3_test, predicted_target3))
      
print('F1 score')
print(sklearn.metrics.f1_score(target3_test, predicted_target3))

Confusion matrix
[[9506 2459]
 [2941 3759]]
Accuracy score
0.7106884543262791
Precision score
0.6045352203280797
Recall score
0.561044776119403
F1 score
0.5819786344635391


### 2.1 Complement Naive Bayes (CNB) T1 (Deaths) <---Model with highest Recall!

CNB is an adaptation of the standard multinomial naive Bayes (MNB) algorithm that is particularly suited for imbalanced data sets, where the algorithm uses statistics from the complement of each class to compute the model’s weight. 

https://scikit-learn.org/stable/modules/naive_bayes.html

Categorize continuous features:

In [76]:
import mlpipeline as pipeline

df_c = df.loc[:,:]
def label_age (row):
    if row['edad'] >= 0  and row['edad'] <= 17 :
        return '0-17'
    if row['edad'] >= 18 and row['edad'] <= 44 :
        return '18-44'
    if row['edad'] >= 45 and row['edad'] <= 64 :
        return '45-64'
    if row['edad'] >= 65 and row['edad'] <= 74 :
        return '65-74'
    return '75 +'

df_c['age_group'] = df_c.apply (lambda row: label_age(row), axis=1)
df_c['pobreza'] = np.where(df_c['pobreza']>=df_c['pobreza'].mean(), 1, 0)
df_c['Densidad_pob'] = np.where(df_c['Densidad_pob']>=df_c['Densidad_pob'].mean(), 1, 0)
df_c['medicos'] = np.where(df_c['medicos']>=df_c['medicos'].mean(), 1, 0)
df_c['camas_hosp'] = np.where(df_c['camas_hosp']>=df_c['camas_hosp'].mean(), 1, 0)
df_c['camas_hosp'] = np.where(df_c['enfermeras']>=df_c['enfermeras'].mean(), 1, 0)

train_c, test_c = pipeline.split_data(df_c.loc[:,'embarazo':], .20, seed=0)

feats_train_c = train_c.loc[:, feats]
target1_train_c = train_c.loc[:, 'muertos']
target2_train_c = train_c.loc[:, 'hospitalizado']
target3_train_c = train_c.loc[:, 'covid_grave']

feats_test_c = test_c.loc[:, feats]
target1_test_c = test_c.loc[:, 'muertos']
target2_test_c = test_c.loc[:, 'hospitalizado']
target3_test_c = test_c.loc[:, 'covid_grave']

In [77]:
from sklearn.naive_bayes import ComplementNB

params = {}

nbg = GridSearchCV(estimator=ComplementNB(), param_grid=params, cv=10 ,
                   return_train_score=True,
                   scoring=['accuracy', 'precision', 'recall'], 
                   refit='accuracy')

nbg.fit(feats_train_c, target1_train_c)
cv_results_nbg = pd.DataFrame(nbg.cv_results_)
cols = ['params', 'mean_test_accuracy', 'mean_test_precision', 'mean_test_recall']

cv_results_nbg[cols]

,params,mean_test_accuracy,mean_test_precision,mean_test_recall
0,{},0.536539,0.15327,0.723745


In [79]:
predicted_target1_c = nbg.predict(feats_test_c)

print('Confusion matrix')
print(sklearn.metrics.confusion_matrix(target1_test_c, predicted_target1_c))

print('Accuracy score')
print(sklearn.metrics.accuracy_score(target1_test_c, predicted_target1_c))

print('Precision score')
print(sklearn.metrics.precision_score(target1_test_c, predicted_target1_c))
      
print('Recall score')
print(sklearn.metrics.recall_score(target1_test_c, predicted_target1_c))
      
print('F1 score')
print(sklearn.metrics.f1_score(target1_test_c, predicted_target1_c))

Confusion matrix
[[8544 8080]
 [ 528 1513]]
Accuracy score
0.5388159657112243
Precision score
0.15771917022829146
Recall score
0.7413032827045566
F1 score
0.26009970775313734


#### SMOTE

In [80]:
pipeline = make_pipeline((SMOTE(random_state=0)), ComplementNB())

kf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

nbg = GridSearchCV(estimator=pipeline, param_grid=params, cv=kf ,
                   return_train_score=True,
                   scoring=['accuracy', 'precision', 'recall'], 
                   refit='accuracy')

nbg.fit(feats_train_c, target1_train_c)
cv_results_nbg = pd.DataFrame(nbg.cv_results_)
cols = ['params', 'mean_test_accuracy', 'mean_test_precision', 'mean_test_recall']

cv_results_nbg[cols]

,params,mean_test_accuracy,mean_test_precision,mean_test_recall
0,{},0.53791,0.153538,0.7228


In [81]:
predicted_target1_c = nbg.predict(feats_test_c)

print('Confusion matrix')
print(sklearn.metrics.confusion_matrix(target1_test_c, predicted_target1_c))

print('Accuracy score')
print(sklearn.metrics.accuracy_score(target1_test_c, predicted_target1_c))

print('Precision score')
print(sklearn.metrics.precision_score(target1_test_c, predicted_target1_c))
      
print('Recall score')
print(sklearn.metrics.recall_score(target1_test_c, predicted_target1_c))
      
print('F1 score')
print(sklearn.metrics.f1_score(target1_test_c, predicted_target1_c))

Confusion matrix
[[8593 8031]
 [ 533 1508]]
Accuracy score
0.5411733190463435
Precision score
0.1580878498794423
Recall score
0.7388535031847133
F1 score
0.26044905008635577


### 2.2 CNB T2 (Hospitalization)

In [82]:
nbg = GridSearchCV(estimator=ComplementNB(), param_grid=params, cv=10 ,
                   return_train_score=True,
                   scoring=['accuracy', 'precision', 'recall'], 
                   refit='accuracy')

nbg.fit(feats_train_c, target2_train_c)
cv_results_nbg = pd.DataFrame(nbg.cv_results_)
cols = ['params', 'mean_test_accuracy', 'mean_test_precision', 'mean_test_recall']

cv_results_nbg[cols]

,params,mean_test_accuracy,mean_test_precision,mean_test_recall
0,{},0.587008,0.443112,0.714609


In [83]:
predicted_target2_c = nbg.predict(feats_test_c)

print('Confusion matrix')
print(sklearn.metrics.confusion_matrix(target2_test_c, predicted_target2_c))

print('Accuracy score')
print(sklearn.metrics.accuracy_score(target2_test_c, predicted_target2_c))

print('Precision score')
print(sklearn.metrics.precision_score(target2_test_c, predicted_target2_c))
      
print('Recall score')
print(sklearn.metrics.recall_score(target2_test_c, predicted_target2_c))
      
print('F1 score')
print(sklearn.metrics.f1_score(target2_test_c, predicted_target2_c))

Confusion matrix
[[6312 5854]
 [1847 4652]]
Accuracy score
0.5874095901419769
Precision score
0.4427945935655816
Recall score
0.7158024311432528
F1 score
0.5471331961187886


#### SMOTE

In [84]:
pipeline = make_pipeline((SMOTE(random_state=0)), ComplementNB())

kf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

nbg = GridSearchCV(estimator=pipeline, param_grid=params, cv=kf ,
                   return_train_score=True,
                   scoring=['accuracy', 'precision', 'recall'], 
                   refit='accuracy')

nbg.fit(feats_train_c, target2_train_c)
cv_results_nbg = pd.DataFrame(nbg.cv_results_)
cols = ['params', 'mean_test_accuracy', 'mean_test_precision', 'mean_test_recall']

cv_results_nbg[cols]

,params,mean_test_accuracy,mean_test_precision,mean_test_recall
0,{},0.587021,0.443114,0.714519


In [85]:
predicted_target2_c = nbg.predict(feats_test_c)

print('Confusion matrix')
print(sklearn.metrics.confusion_matrix(target2_test_c, predicted_target2_c))

print('Accuracy score')
print(sklearn.metrics.accuracy_score(target2_test_c, predicted_target2_c))

print('Precision score')
print(sklearn.metrics.precision_score(target2_test_c, predicted_target2_c))
      
print('Recall score')
print(sklearn.metrics.recall_score(target2_test_c, predicted_target2_c))
      
print('F1 score')
print(sklearn.metrics.f1_score(target2_test_c, predicted_target2_c))

Confusion matrix
[[6326 5840]
 [1854 4645]]
Accuracy score
0.5877846236271096
Precision score
0.4430138292799237
Recall score
0.7147253423603631
F1 score
0.5469853980216675


### 2.3 CNB T3 (Hospitalization/Death)

In [87]:
nbg = GridSearchCV(estimator=ComplementNB(), param_grid=params, cv=10 ,
                   return_train_score=True,
                   scoring=['accuracy', 'precision', 'recall'], 
                   refit='accuracy')

nbg.fit(feats_train_c, target3_train_c)
cv_results_nbg = pd.DataFrame(nbg.cv_results_)
cols = ['params', 'mean_test_accuracy', 'mean_test_precision', 'mean_test_recall']

cv_results_nbg[cols]

,params,mean_test_accuracy,mean_test_precision,mean_test_recall
0,{},0.591776,0.457447,0.717346


In [88]:
predicted_target3_c = nbg.predict(feats_test_c)

print('Confusion matrix')
print(sklearn.metrics.confusion_matrix(target3_test_c, predicted_target3_c))

print('Accuracy score')
print(sklearn.metrics.accuracy_score(target3_test_c, predicted_target3_c))

print('Precision score')
print(sklearn.metrics.precision_score(target3_test_c, predicted_target3_c))
      
print('Recall score')
print(sklearn.metrics.recall_score(target3_test_c, predicted_target3_c))
      
print('F1 score')
print(sklearn.metrics.f1_score(target3_test_c, predicted_target3_c))

Confusion matrix
[[6230 5735]
 [1897 4803]]
Accuracy score
0.5911063487811412
Precision score
0.4557790852154109
Recall score
0.716865671641791
F1 score
0.5572572224155935


#### SMOTE

In [89]:
pipeline = make_pipeline((SMOTE(random_state=0)), ComplementNB())

kf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

nbg = GridSearchCV(estimator=pipeline, param_grid=params, cv=kf ,
                   return_train_score=True,
                   scoring=['accuracy', 'precision', 'recall'], 
                   refit='accuracy')

nbg.fit(feats_train_c, target3_train_c)
cv_results_nbg = pd.DataFrame(nbg.cv_results_)
cols = ['params', 'mean_test_accuracy', 'mean_test_precision', 'mean_test_recall']

cv_results_nbg[cols]

,params,mean_test_accuracy,mean_test_precision,mean_test_recall
0,{},0.591388,0.457128,0.717284


In [90]:
predicted_target3_c = nbg.predict(feats_test_c)

print('Confusion matrix')
print(sklearn.metrics.confusion_matrix(target3_test_c, predicted_target3_c))

print('Accuracy score')
print(sklearn.metrics.accuracy_score(target3_test_c, predicted_target3_c))

print('Precision score')
print(sklearn.metrics.precision_score(target3_test_c, predicted_target3_c))
      
print('Recall score')
print(sklearn.metrics.recall_score(target3_test_c, predicted_target3_c))
      
print('F1 score')
print(sklearn.metrics.f1_score(target3_test_c, predicted_target3_c))

Confusion matrix
[[6231 5734]
 [1891 4809]]
Accuracy score
0.5914813822662738
Precision score
0.4561320307312909
Recall score
0.7177611940298507
F1 score
0.5577915675926464
